In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import matplotlib.pyplot as plt
import model.cell as cell
import model.spec as spec
import numpy as np


(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [2]:
np.shape(train_images)

(50000, 32, 32, 3)

In [3]:

cf = spec.Spec.get_configuration_space().sample_configuration()
s = spec.Spec(cf)
while s.valid_spec == False:
    cf = spec.Spec.get_configuration_space().sample_configuration()
    s = spec.Spec(cf)

i = tf.keras.Input(shape=(32, 32, 3))
c = cell.Cell(s, 128, i).build()
c = cell.Cell(s, 128, c).build()
c = cell.Cell(s, 128, c).build()
c = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same')(c)
c = cell.Cell(s, 256, i).build()
c = cell.Cell(s, 256, c).build()
c = cell.Cell(s, 256, c).build()
c = tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2), padding='same')(c)
c = cell.Cell(s, 512, i).build()
c = cell.Cell(s, 512, c).build()
c = cell.Cell(s, 512, c).build()
c = tf.reduce_mean(c, [1,2])
o = tf.keras.layers.Flatten()(c)
o = tf.keras.layers.Dense(64, activation='softmax')(o)
o = tf.keras.layers.Dense(10)(o)
model = tf.keras.Model(i, o, name="sample_model")
model.summary()

print(s.matrix)
print(s.ops)

Model: "sample_cell"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 32, 32, 256)  768         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 32, 32, 256)  1024        conv2d_18[0][0]                  
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 32, 32, 256)  768         input_1[0][0]                    
________________________________________________________________________________________

In [12]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
history = model.fit(train_images, train_labels, epochs=5, validation_data=(test_images, test_labels))

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 119s 2ms/sample - loss: 2.0220 - accuracy: 0.2327 - val_loss: 2.0005 - val_accuracy: 0.2232
Epoch 2/5
50000/50000 [==============================] - 118s 2ms/sample - loss: 1.7944 - accuracy: 0.2762 - val_loss: 1.7252 - val_accuracy: 0.3222
Epoch 3/5
50000/50000 [==============================] - 118s 2ms/sample - loss: 1.6495 - accuracy: 0.3536 - val_loss: 1.5758 - val_accuracy: 0.3860
Epoch 4/5
50000/50000 [==============================] - 118s 2ms/sample - loss: 1.5153 - accuracy: 0.3996 - val_loss: 1.5557 - val_accuracy: 0.3832
Epoch 5/5
50000/50000 [==============================] - 118s 2ms/sample - loss: 1.4271 - accuracy: 0.4275 - val_loss: 1.4736 - val_accuracy: 0.4069
